# Visualising Predictions

### 1. Imports and Model Initialization

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from scripts.utils import FinalModel, visualize_prediction
from scripts.config_io import save_to_config, get_config_value
from transformers import RobertaModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base",add_prefix_space=True)
text_encoder = RobertaModel.from_pretrained("roberta-base")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
choices = ["final_weights.pth","best_weights.pth"]      #Choose
i = 0
model_save_weight_path = None
try:
    Path(get_config_value("CHECKPOINTS_PATH",default="not-set")) / choices[i]
except FileNotFoundError as e:
    print(f"Please provide the link for the {choices[i]}")
    model_save_weight_path = Path(input())
    save_to_config({
        "CHECKPOINTS_PATH":str(model_save_weight_path.parent),
    })

In [ ]:
model = FinalModel()
state_dict = torch.load(model_save_weight_path)
model.load_state_dict(state_dict)
model.eval()

## 3. Evaluate

In [ ]:
phrase = ""
img_path : Path = Path(get_config_value("OUT_DIR",default="not-set")) / ""      #Fill with your folder first and then your 

In [ ]:
#Run this cell to see the image before predicted boxes
img = Image.open(img_path).convert("RGB")
img_np = np.array(img)
plt.imshow(img_np)
plt.axis('off')
plt.title("The image")
plt.show()

In [ ]:
visualize_prediction(model,img_path,phrase,tokenizer,device)